<p style="font-weight:bold;font-size:200%"> Exclude Liquidity Provision </p>

- Author: Yichen Luo
- Date: 20220819

In [1]:
import time
import random
import numpy as np
import pandas as pd
from datetime import timezone
import datetime
import tweepy
import requests
import configparser
import matplotlib.pyplot as plt
import re
import os
from bs4 import BeautifulSoup
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
RD = '/home/team_defi/02_rdata/'
WD = '/home/team_defi/03_wdata/'
RES = '/home/team_defi/04_result/'

# Exclude

In [25]:
df = pd.read_csv(f'{RES}/insider_trading/token_related_txn/token_related_txn.csv')
df['lp'] = np.nan
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
num = 0
length = df.drop_duplicates('transaction_id').shape[0]

for idx, row in df.drop_duplicates('transaction_id').iterrows():
    try:
        num += 1
        transaction_id = row['transaction_id']
        chain = row['chain']
    
        if chain == 'ethereum': url = f'https://etherscan.io/tx/{transaction_id}'
        if chain == 'binance' : url = f'https://bscscan.com/tx/{transaction_id}'
        if chain == 'avalanche': url = f'https://snowtrace.io/tx/{transaction_id}'
        if chain == 'polygon' : url = f'https://polygonscan.com/tx/{transaction_id}'
        if chain == 'fantom' : url = f'https://ftmscan.com/tx/{transaction_id}'

        r = requests.get(url, headers=headers)
        if r.status_code != 200 : continue
        if re.search('liquidity to', r.text.lower()) != None:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = True
            print(f'{chain}:{transaction_id}:lp, {num}/{length}')
        else:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = False
            print(f'{chain}:{transaction_id}:normal, {num}/{length}')
        
        time.sleep(2 + random.random() * 2)
    except Exception as e:
        print(f'{chain}:{transaction_id}:error{e}, {num}/{length}')
        time.sleep(5)

ethereum:0xe91b835550a1a50b26a0646fa3e1def942a78ea5ce7349529a30700eff08ab11:normal, 1/15119
ethereum:0xd1644c62998cbb5fa03b412ab9c62cd560706e04d2399fd86860fcd10159c499:normal, 2/15119
ethereum:0x099a3083fd9a904646021f522bef9d887e6ab07b5171a59ca278c4edce13c43b:normal, 3/15119
ethereum:0x2f9ee0617b0024f000eda6cef85507776e2b7d2f2f637858764eb2c652f6c40e:normal, 4/15119
ethereum:0x8e7be63d557448574c91c0ff62e26339473a08906565e79a034db46ac67dea95:normal, 5/15119
ethereum:0xeabdfd5a21391f7d7325aa00751e3b292e4df4572920f06ca0d09e1d13a9a34c:normal, 6/15119
ethereum:0x812454dcafc0ec808ccae7a55357e034d3f1cf5fc515c59c7c4d0c0613c200dc:normal, 7/15119
ethereum:0x56b45d69902cb3a450a7ce22c3915bed2880e03221f5635cff15d2ef04451a31:normal, 8/15119
ethereum:0x155ae95a7908613b6382f6e069ea0e7177d6ec6f9a9b2545a6700afe61e70f76:normal, 9/15119
ethereum:0xcabbe3101b1f22dd9a8b1629088266d2129e0386060a3c8fcc8c3df896836490:normal, 10/15119
ethereum:0xd922a7f36604a7b5f986fa1d83046bef8ee4916147bcf6598d2a30580835433e:nor

# 处理Error

In [39]:
df.loc[df['lp'].isnull()]
df.loc[df['transaction_id'] == '0xcdcce475561ca7bf0373be07acadccb42f75db299e7f0c7f4eace69f46597af9']

,id,amountUSD,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,insider_root,insider_tokenName,insider_tokenSymbol,insider_contractAddress,dex,chain,amount0Out,amount0In,amount1Out,amount1In,from,to,contract,buy,lp


,id,amountUSD,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,insider_root,insider_tokenName,insider_tokenSymbol,insider_contractAddress,dex,chain,amount0Out,amount0In,amount1Out,amount1In,from,to,contract,buy,lp
16316,0xcdcce475561ca7bf0373be07acadccb42f75db299e7f0c7f4eace69f46597af9-0,0.283798,201,NaN,NaN,0xa5e0829caced8ffdd4de3c43696c57f7d7a678ff,NaN,NaN,2022-05-13 12:14:55,Wrapped Matic,NextEarthToken,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x374af2db27a0c655beba4323b4c4273dc30eba73,0xcdcce475561ca7bf0373be07acadccb42f75db299e7f0c7f4eace69f46597af9,0x5a6dc942826807539ca19d21c132e4b004075de0,NextEarthToken,NXTT,0x0d0B8488222F7f83B23E365320a4021b12eAD608,quickswap_polygon,polygon,0.0,0.4,34942.260091,0.0,0x5a6dc942826807539ca19d21c132e4b004075de0,0x5a6dc942826807539ca19d21c132e4b004075de0,False,0.283798,False


In [36]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
num = 0
length = df.loc[df['lp'].isnull()].shape[0]

for idx, row in df.loc[df['lp'].isnull()].iterrows():
    try:
        num += 1
        transaction_id = row['transaction_id']
        chain = row['chain']
    
        if chain == 'ethereum': url = f'https://etherscan.io/tx/{transaction_id}'
        if chain == 'binance' : url = f'https://bscscan.com/tx/{transaction_id}'
        if chain == 'avalanche': url = f'https://snowtrace.io/tx/{transaction_id}'
        if chain == 'polygon' : url = f'https://polygonscan.com/tx/{transaction_id}'
        if chain == 'fantom' : url = f'https://ftmscan.com/tx/{transaction_id}'

        r = requests.get(url, headers=headers)
        if r.status_code != 200 : continue
        if re.search('liquidity to', r.text.lower()) != None:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = True
            print(f'{chain}:{transaction_id}:lp, {num}/{length}')
        else:
            df.loc[df['transaction_id'] == transaction_id, 'lp'] = False
            print(f'{chain}:{transaction_id}:normal, {num}/{length}')
        
        time.sleep(2 + random.random() * 2)
    except Exception as e:
        print(f'{chain}:{transaction_id}:error{e}, {num}/{length}')
        time.sleep(5)

binance:0x4a992b713fbb41308052e9e1b641c7e6745242ffa520351081776e8a1f3955b0:normal, 1/10
avalanche:0x692ff69c9ad24077527399843bcbbe3fe4a43581f39b72087140d9514821083d:normal, 2/10
polygon:0xd46e06352ff371e4c3905032506f9da07d6c44cb078f03fcbe6b792d8283e010:normal, 3/10
polygon:0x90afc8e6bddef741502753aaee4c2b490cfc05c870b850a2261dcc8fe5eff9de:normal, 4/10
polygon:0x73637883388090a1beb450fedc08e92979b715a397290ec8d21e1bda6e1e53a8:normal, 5/10
polygon:0xd02bf94cdb34621d9d4d3a2ae64c2a34a63758f4c098951a869940bec3ec35e9:normal, 6/10
polygon:0xcdf1ad75d0ce61212b1eeb152280835492cd09f95693507a0933ecd28950399c:normal, 7/10
polygon:0x10fa3836c05f54933ec34baeed2641cc69bd1f2ee142e596991abc1922f0f81d:normal, 8/10
polygon:0x697c7197dbd983c8ff6b965be6ca28c45da467f0f22ab4ddb98b28fc9e7361c1:normal, 9/10
polygon:0xcdcce475561ca7bf0373be07acadccb42f75db299e7f0c7f4eace69f46597af9:normal, 10/10


In [40]:
# df.to_csv(f'{RES}/insider_trading/token_related_txn/token_related_txn_lp.csv', index=False)

# 输出

In [45]:
df = pd.read_csv(f'{RES}/insider_trading/token_related_txn/token_related_txn_lp.csv')
df.shape[0]
df.loc[df['lp'].isnull()]

df = df.loc[df['lp'] == False]
df.shape[0]

16410

,id,amountUSD,logIndex,origin,recipient,sender,sqrtPriceX96,tick,timestamp,token0_name,token1_name,token0_id,token1_id,transaction_id,insider_root,insider_tokenName,insider_tokenSymbol,insider_contractAddress,dex,chain,amount0Out,amount0In,amount1Out,amount1In,from,to,contract,buy,lp


14716

In [43]:
df.to_csv(f'{RES}/insider_trading/token_related_txn/token_related_txn_without_lp.csv', index=False)

# Debug

In [18]:
transaction_id = '0x920f29850b3a182809b8437978109c6318fd5ab381758dcad4d01406ff033000'
chain = 'ethereum'

if chain == 'ethereum': url = f'https://etherscan.io/tx/{transaction_id}'
if chain == 'binance' : url = f'https://bscscan.com/tx/{transaction_id}'
if chain == 'avalanche': url = f'https://snowtrace.io/tx/{transaction_id}'
if chain == 'polygon' : url = f'https://polygonscan.com/tx/{transaction_id}'
if chain == 'fantom' : url = f'https://ftmscan.com/tx/{transaction_id}'

r = requests.get(url, headers=headers)
bs = BeautifulSoup(r.text)

In [24]:
re.search('liquidity to', r.text.lower()) != None

True